Actividad 2.3: Lasso - Ridge - ElasticNet Regressors
=
### Jorge Emilio Treviño González - #545388
### Jaime
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# Librería de significancia estadística
import statsmodels.formula.api as smf

# Librería para separar los datos en train y tes
from sklearn.model_selection import train_test_split

# Librería del modelo de regresión lineal 
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error as mse #MSE 
from sklearn.metrics import mean_absolute_percentage_error as mape #MAPE

# Librería para optimizar parámetros
from sklearn.model_selection import GridSearchCV

In [2]:
# Leer los datos
df = pd.read_csv('Walmart.csv')
print(df.shape)
df.head(3)
#Store

(6435, 8)


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106


__Ingeniería de datos básica y elemental__

In [3]:
# Identificando datos vacíos y registros duplicados. 
print("Tabla de datos vacios por columna:\n")
print(df.isna().sum())
print("\nLa cantidad de registros duplicados es de:",df.duplicated(subset = df.columns, keep = False).sum())
print("\nTabla de registros duplicados:")
df[df.duplicated(subset = df.columns, keep = False)]

Tabla de datos vacios por columna:

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

La cantidad de registros duplicados es de: 0

Tabla de registros duplicados:


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment


_La base de datos se encuentra completa y sin registros duplicados_

In [4]:
df.dtypes

Store             int64
Date             object
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [5]:
# Convirtiendo la variable object en int
df['date_num'] = df['Date'].astype('category').cat.codes
df.head(3)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,date_num
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106,19
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106,52
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106,85


__Estudiamos la correlación__

In [6]:
df.corr().sort_values(by = 'Weekly_Sales')

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,date_num
Store,1.000000e+00,-0.335332,-4.386841e-16,-0.022659,0.060023,-0.209492,0.223531,-1.600195e-15
Unemployment,2.235313e-01,-0.106176,1.096028e-02,0.101158,-0.034684,-0.302020,1.000000,-4.038805e-03
CPI,-2.094919e-01,-0.072634,-2.162091e-03,0.176888,-0.170642,1.000000,-0.302020,2.593192e-03
Temperature,-2.265908e-02,-0.063810,-1.550913e-01,1.000000,0.144982,0.176888,0.101158,3.466788e-02
date_num,-1.600195e-15,-0.015453,4.516981e-02,0.034668,0.023262,0.002593,-0.004039,1.000000e+00
Fuel_Price,6.002295e-02,0.009464,-7.834652e-02,0.144982,1.000000,-0.170642,-0.034684,2.326200e-02
Holiday_Flag,-4.386841e-16,0.036891,1.000000e+00,-0.155091,-0.078347,-0.002162,0.010960,4.516981e-02
Weekly_Sales,-3.353320e-01,1.000000,3.689097e-02,-0.063810,0.009464,-0.072634,-0.106176,-1.545254e-02


__Future Selection__ - ANOVA
<!--  -->
Usaremos la significancia estadística (prueba de hipótesis)

In [7]:
FS = smf.ols(formula = 'Weekly_Sales ~ Store + Holiday_Flag + Temperature + Fuel_Price + CPI + Unemployment + date_num',
            data = df).fit()
FS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Weekly_Sales   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     151.7
Date:                Sun, 09 Oct 2022   Prob (F-statistic):          5.02e-208
Time:                        23:26:46   Log-Likelihood:                -93860.
No. Observations:                6435   AIC:                         1.877e+05
Df Residuals:                    6427   BIC:                         1.878e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2.009e+06    7.6e+04     26.424      0.000    1.86e+06    2.16e+06
Store        -1.539e+04    521.859    -29.487      0.000   -1.64e+04   -1.44e+04
Holiday_Flag  7.492e+04    2.6e+04      2.884      0.004     2.4e+04    1.26e+05
Temperature   -955.0561    376.237     -2.538      0.011   -1692.607    -217.505
Fuel_Price    9995.5107   1.48e+04      0.675      0.500    -1.9e+04     3.9e+04
CPI          -2320.3359    184.760    -12.559      0.000   -2682.527   -1958.145
Unemployment -2.193e+04   3787.880     -5.789      0.000   -2.94e+04   -1.45e+04
date_num      -218.2346    158.317     -1.378      0.168    -528.589      92.120
==============================================================================
Omnibus:                      191.096   Durbin-Watson:                   0.128
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              207.735
Skew:                           0.436   Prob(JB):                     7.78e-46
Kurtosis:                       3.114   Cond. No.                     2.36e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.36e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
# Obtenemos los pvalues
FS.pvalues.sort_values(ascending = True)

Store           2.403443e-179
Intercept       3.683003e-146
CPI              9.346466e-36
Unemployment     7.427731e-09
Holiday_Flag     3.939325e-03
Temperature      1.115818e-02
date_num         1.681082e-01
Fuel_Price       4.998148e-01
dtype: float64

Las variables que tienen un *p_value* mayor a 0.05, y por tanto __no son importantes__ para el modelo son:
* '*date_num*'
* '*Fuel_price*'

In [9]:
FS = smf.ols(formula = 'Weekly_Sales ~ Store + CPI + Unemployment + Holiday_Flag + Temperature',
            data = df).fit()
FS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Weekly_Sales   R-squared:                       0.141
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     211.9
Date:                Sun, 09 Oct 2022   Prob (F-statistic):          7.51e-210
Time:                        23:26:46   Log-Likelihood:                -93861.
No. Observations:                6435   AIC:                         1.877e+05
Df Residuals:                    6429   BIC:                         1.878e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     2.032e+06   5.07e+04     40.114      0.000    1.93e+06    2.13e+06
Store        -1.537e+04    521.337    -29.488      0.000   -1.64e+04   -1.44e+04
CPI          -2345.9264    180.191    -13.019      0.000   -2699.160   -1992.693
Unemployment  -2.22e+04   3755.948     -5.910      0.000   -2.96e+04   -1.48e+04
Holiday_Flag  7.222e+04   2.59e+04      2.787      0.005    2.14e+04    1.23e+05
Temperature   -929.0252    369.081     -2.517      0.012   -1652.547    -205.503
==============================================================================
Omnibus:                      188.685   Durbin-Watson:                   0.130
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              204.924
Skew:                           0.434   Prob(JB):                     3.17e-45
Kurtosis:                       3.100   Cond. No.                     1.46e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.46e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

_Sigue habiendo presencia de multicolinealidad_

__Detectar multicolinealidad calculando el VIF__
* VIF con valores de 1 (_el menor valor de VIF_) significa que la variable predictora no se relaciona con las demás variables predictoras
* Mientras más grande sea el VIF Mayor correlación tiene la variabe predictora con otras variables predictoras

In [10]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X_variables = df[['Store','Holiday_Flag','Temperature','Fuel_Price', 'CPI', 'Unemployment', 'date_num']]
vif_data = pd.DataFrame()
vif_data["feature"] = X_variables.columns
vif_data["VIF"] = [variance_inflation_factor(X_variables.values, i) for i in range(len(X_variables.columns))]
vif_data.sort_values(by = 'VIF', ascending = False)

,feature,VIF
3,Fuel_Price,27.496157
5,Unemployment,16.385614
4,CPI,14.935729
2,Temperature,13.382740
0,Store,4.403808
6,date_num,3.911741
1,Holiday_Flag,1.104054


In [11]:
FS = smf.ols(formula = 'Weekly_Sales ~ Store + Holiday_Flag + Temperature', data = df).fit()
FS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Weekly_Sales   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     287.4
Date:                Sun, 09 Oct 2022   Prob (F-statistic):          4.27e-175
Time:                        23:26:46   Log-Likelihood:                -93947.
No. Observations:                6435   AIC:                         1.879e+05
Df Residuals:                    6431   BIC:                         1.879e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     1.505e+06   2.63e+04     57.159      0.000    1.45e+06    1.56e+06
Store        -1.464e+04    508.938    -28.760      0.000   -1.56e+04   -1.36e+04
Holiday_Flag  5.852e+04   2.62e+04      2.231      0.026    7105.536     1.1e+05
Temperature  -2060.4647    362.763     -5.680      0.000   -2771.602   -1349.328
==============================================================================
Omnibus:                      138.026   Durbin-Watson:                   0.126
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              146.686
Skew:                           0.370   Prob(JB):                     1.40e-32
Kurtosis:                       3.024   Cond. No.                         293.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

_Se logró __eliminar la multicolinealidad___

__Las variables más importantes para explicar las ventas semanales son:__
1. '_Store_'
2. '_Temperature_'
3. '_Holiday_Flag_'

In [12]:
# Escogemos las variables a usar
x = df[['Store', 'Temperature', 'Holiday_Flag']] #-> doble corchete para tener la columna en datos de dos dimensiones (como los necesita python para la regresión)
y = df[['Weekly_Sales']]
print(x.shape)
print(y.shape)

(6435, 3)
(6435, 1)


In [13]:
# Dividimos los datos en train y test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

#Ver los resultados
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((5148, 3), (1287, 3), (5148, 1), (1287, 1))

In [14]:
# Definimos el modelo de RLM
RLM = LinearRegression()

# Entrenamos el modelo
RLM.fit(x_train, y_train)


LinearRegression()

In [15]:
# Validación visual
y_pred_test = RLM.predict(x_test)
print(y_pred_test[0:5])
y_test.head()

[[ 957311.15127964]
 [ 892890.63880441]
 [1177198.21293455]
 [ 834237.33769904]
 [1043358.84443094]]


,Weekly_Sales
3949,1220984.94
4555,1246322.44
2235,505918.21
5204,527572.25
3524,719235.07


__Evaluamos el modelo:__

In [16]:
# Obtieniento el R2 de train
R2_train = RLM.score(x_train,y_train)
print("R2 train = {:.2f}".format(R2_train))

# Obtieniento el R2 de test
R2_test = RLM.score(x_test,y_test)
print("R2 test = {:.2f}".format(R2_test))

# Diferencia
dif = '{:.4}%'.format(np.abs(R2_train - R2_test)*100)
print("Diferencia = ",dif)

R2 train = 0.11
R2 test = 0.14
Diferencia =  3.352%


_¡El modelo __no__ tiene overfitting!_

__Precisión del modelo:__

In [17]:
# Obtener el y_pred_train
y_pred_train = RLM.predict(x_train)

# Calculamos los errores de train
mape_train = mape(y_train, y_pred_train)*100
mse_train = mse(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)

# Calculamos los errores de test
mape_test = mape(y_test, y_pred_test)*100
mse_test = mse(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)

print("MAPE de train = {:.2}%".format(mape_train))
print("MSE de train = {:.2}".format(mse_train))
print("RMSE de train = {:.2}".format(rmse_train))
print("")
print("MAPE de test = {:.2}%".format(mape_test))
print("MSE de test = {:.2}".format(mse_test))
print("RMSE de test = {:.2}".format(rmse_test))

MAPE de train = 6.2e+01%
MSE de train = 4.4e+05
RMSE de train = 6.6e+02

MAPE de test = 6e+01%
MSE de test = 4.2e+05
RMSE de test = 6.5e+02


__Datos del modelo:__

In [18]:
print("Datos del modelo de Regresión Lineal Múltiple")
print()
print("B0 = {:.3}".format(RLM.intercept_[0]))
print("B1 = {:.3}".format(RLM.coef_[0][0]))
print("B2 = {:.3}".format(RLM.coef_[0][1]))
print("B3 = {:.3}".format(RLM.coef_[0][2]))
print()
print("La ecuacón del modelo es:")
print("y_pred = {:.3} + {:.3}x1 + {:.3}x2 + {:.3}x3".format(RLM.intercept_[0], RLM.coef_[0][0], RLM.coef_[0][1], RLM.coef_[0][2]))

Datos del modelo de Regresión Lineal Múltiple

B0 = 1.49e+06
B1 = -1.43e+04
B2 = -1.92e+03
B3 = 5.35e+04

La ecuacón del modelo es:
y_pred = 1.49e+06 + -1.43e+04x1 + -1.92e+03x2 + 5.35e+04x3


Lasso
=
**L**east **A**bsolute **S**hrinkage and **S**election **O**perator

In [19]:
from sklearn.linear_model import Lasso

In [20]:
df_lasso = pd.read_csv("Walmart.csv")
df_lasso['date_num'] = df_lasso['Date'].astype('category').cat.codes

In [21]:
x_lasso = df_lasso.drop(columns=['Weekly_Sales','Date'])
y_lasso = df_lasso['Weekly_Sales']
print(x_lasso.shape)
print(y_lasso.shape)

(6435, 7)
(6435,)


In [22]:
# Dividir los datos en train y test
x_train_lasso, x_test_lasso, y_train_lasso, y_test_lasso = train_test_split(x, y, test_size = 0.2, random_state = 0)
x_train_lasso.shape, x_test_lasso.shape, y_train_lasso.shape, y_test_lasso.shape

((5148, 3), (1287, 3), (5148, 1), (1287, 1))

In [23]:
# Definir el rango de variación de los parámetros
param_range = {'alpha': np.arange(30, 100.25, 0.25),
               'max_iter': range(1, 50, 1)}

# Veamos los rangos a usar
print(list(param_range['alpha']))
print(list(param_range['max_iter']))

[30.0, 30.25, 30.5, 30.75, 31.0, 31.25, 31.5, 31.75, 32.0, 32.25, 32.5, 32.75, 33.0, 33.25, 33.5, 33.75, 34.0, 34.25, 34.5, 34.75, 35.0, 35.25, 35.5, 35.75, 36.0, 36.25, 36.5, 36.75, 37.0, 37.25, 37.5, 37.75, 38.0, 38.25, 38.5, 38.75, 39.0, 39.25, 39.5, 39.75, 40.0, 40.25, 40.5, 40.75, 41.0, 41.25, 41.5, 41.75, 42.0, 42.25, 42.5, 42.75, 43.0, 43.25, 43.5, 43.75, 44.0, 44.25, 44.5, 44.75, 45.0, 45.25, 45.5, 45.75, 46.0, 46.25, 46.5, 46.75, 47.0, 47.25, 47.5, 47.75, 48.0, 48.25, 48.5, 48.75, 49.0, 49.25, 49.5, 49.75, 50.0, 50.25, 50.5, 50.75, 51.0, 51.25, 51.5, 51.75, 52.0, 52.25, 52.5, 52.75, 53.0, 53.25, 53.5, 53.75, 54.0, 54.25, 54.5, 54.75, 55.0, 55.25, 55.5, 55.75, 56.0, 56.25, 56.5, 56.75, 57.0, 57.25, 57.5, 57.75, 58.0, 58.25, 58.5, 58.75, 59.0, 59.25, 59.5, 59.75, 60.0, 60.25, 60.5, 60.75, 61.0, 61.25, 61.5, 61.75, 62.0, 62.25, 62.5, 62.75, 63.0, 63.25, 63.5, 63.75, 64.0, 64.25, 64.5, 64.75, 65.0, 65.25, 65.5, 65.75, 66.0, 66.25, 66.5, 66.75, 67.0, 67.25, 67.5, 67.75, 68.0, 68.25

### Optimización de Hiperparámetros - Lasso

In [24]:
# Definir el optimizador
rsearch_lasso = GridSearchCV(Lasso(random_state = 0),
                             param_range,
                             n_jobs = -1 , # Usa todos los nucleos
                             verbose = 4,
                             cv = 3)
# Optimizamos
rsearch_lasso.fit(x_train_lasso,y_train_lasso)

Fitting 3 folds for each of 13769 candidates, totalling 41307 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.268e+14, tolerance: 1.642e+11
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=3, estimator=Lasso(random_state=0), n_jobs=-1,
             param_grid={'alpha': array([ 30.  ,  30.25,  30.5 ,  30.75,  31.  ,  31.25,  31.5 ,  31.75,
        32.  ,  32.25,  32.5 ,  32.75,  33.  ,  33.25,  33.5 ,  33.75,
        34.  ,  34.25,  34.5 ,  34.75,  35.  ,  35.25,  35.5 ,  35.75,
        36.  ,  36.25,  36.5 ,  36.75,  37.  ,  37.25,  37.5 ,  37.75,
        38.  ,  38.25,  38.5 ,  38.75,  39.  ,  39.25,  39.5 ,  39.75,
        40.  ,  40.25,  40.5 ,  40.75,  41.  ,  41.25,  41.5 ,  41.75,
        42.  ,  42.25,  4...
        84.  ,  84.25,  84.5 ,  84.75,  85.  ,  85.25,  85.5 ,  85.75,
        86.  ,  86.25,  86.5 ,  86.75,  87.  ,  87.25,  87.5 ,  87.75,
        88.  ,  88.25,  88.5 ,  88.75,  89.  ,  89.25,  89.5 ,  89.75,
        90.  ,  90.25,  90.5 ,  90.75,  91.  ,  91.25,  91.5 ,  91.75,
        92.  ,  92.25,  92.5 ,  92.75,  93.  ,  93.25,  93.5 ,  93.75,
        94.  ,  94.25,  94.5 ,  94.75,  95.  ,  95.25,  95.5 ,  95.75,
        96.  ,  96.25,  96.5 ,  96.75,  97.  ,  97.25,  97.5 ,  97.75,
        98.  ,  98.25,  98.5 ,  98.75,  99.  ,  99.25,  99.5 ,  99.75,
       100.  ]),
                         'max_iter': range(1, 50)},
             verbose=4)

In [25]:
# Veamos los resultados
print('R2_primdio = ', rsearch_lasso.best_score_)
print('Mejor modelo =', rsearch_lasso.best_estimator_)
print('Mejores parámatros = ', rsearch_lasso.best_params_)

R2_primdio =  0.10858687583326491
Mejor modelo = Lasso(alpha=77.75, max_iter=1, random_state=0)
Mejores parámatros =  {'alpha': 77.75, 'max_iter': 1}


In [26]:
# Definimos el modelo Lasso
lasso = rsearch_lasso.best_estimator_

# Entrenamos el modelo
lasso.fit(x_train_lasso, y_train_lasso)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.268e+14, tolerance: 1.642e+11
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=77.75, max_iter=1, random_state=0)

In [27]:
# Validación visual
y_pred_test_lasso = lasso.predict(x_test_lasso)
print(y_pred_test_lasso[0:5])
y_test_lasso.head()

[ 957016.36820994  892388.83984108 1179118.69090962  834786.38349303
 1045478.55958467]


,Weekly_Sales
3949,1220984.94
4555,1246322.44
2235,505918.21
5204,527572.25
3524,719235.07


In [28]:
print('Slope: %.2f' % lasso.coef_[0])
print('Intercept: %.2f' % lasso.intercept_)

Slope: -14205.95
Intercept: 1491928.88


__Evaluamos el modelo:__

In [29]:
# Obtieniento el R2 de train
R2_train_lasso = lasso.score(x_train,y_train)
print("R2 train de Lasso = {:.2f}".format(R2_train_lasso))

# Obtieniento el R2 de test
R2_test_lasso = lasso.score(x_test,y_test)
print("R2 test de Lasso = {:.2f}".format(R2_test_lasso))

# Diferencia
dif_lasso = '{:.4}%'.format(np.abs(R2_train_lasso - R2_test_lasso)*100)
print("Diferencia = ",dif_lasso)

R2 train de Lasso = 0.11
R2 test de Lasso = 0.14
Diferencia =  3.355%


_¡El modelo __no__ tiene overfitting!_

__Precisión del modelo:__

In [30]:
# Obtener el y_pred_train
y_pred_train_lasso = lasso.predict(x_train_lasso)

# Calculamos los errores de train
mape_train_lasso = mape(y_train_lasso, y_pred_train_lasso)*100
mse_train_lasso = mse(y_train_lasso, y_pred_train_lasso)
rmse_train_lasso = np.sqrt(mse_train_lasso)

# 3. Calculamos los errores de test
mape_test_lasso = mape(y_test_lasso, y_pred_test_lasso)*100
mse_test_lasso = mse(y_test_lasso, y_pred_test_lasso)
rmse_test_lasso = np.sqrt(mse_test_lasso)

print("MAPE de train de Lasso = {:.2}%".format(mape_train_lasso))
print("MSE de train de Lasso = {:.2}".format(mse_train_lasso))
print("RMSE de train de Lasso = {:.2}".format(rmse_train_lasso))
print("")
print("MAPE de test de Lasso = {:.2}%".format(mape_test_lasso))
print("MSE de test de Lasso = {:.2}".format(mse_test_lasso))
print("RMSE de test de Lasso = {:.2}".format(rmse_test_lasso))

MAPE de train de Lasso = 6.2e+01%
MSE de train de Lasso = 4.4e+05
RMSE de train de Lasso = 6.6e+02

MAPE de test de Lasso = 6e+01%
MSE de test de Lasso = 4.2e+05
RMSE de test de Lasso = 6.5e+02


### Ventajas y desventajas de la Regresión Lasso
_Ventajas:_
1. Reduce la complejidad del modelo
2. Elimina (reduciendo a cero) los coeficientes con la ayuda de la selección automática de variables para los modelos
3. Ayuda a resolver modelos con overfitting


_Desventajas:_
1.  Se introduce(n) hiperparámetro(s)
2. Batalla para estimar los errores estándar
3. Ortos problemas asociados con los modelos de regresión

---
### Campo en dónde se utiliza más la Regresión Lasso
El algoritmo de Regresión Lasso también es llamado "Método de Regresión Penalizado" y suele utilizatrse en machine learning para la seleción de subconjuntos de variables. En general , Lasso proporciona una mayor predicción a comparación de otros métodos de regresión y por esto es útil para las interpretaciones de los modelos. 

---

Ridge
==

In [31]:
from sklearn.linear_model import Ridge

In [32]:
df_ridge = pd.read_csv("Walmart.csv")
df_ridge['date_num'] = df_ridge['Date'].astype('category').cat.codes

In [33]:
x_ridge = df_ridge.drop(columns=['Weekly_Sales','Date'])
y_ridge = df_ridge['Weekly_Sales']
print(x_ridge.shape)
print(y_ridge.shape)

(6435, 7)
(6435,)


In [34]:
# Dividir los datos en train y test
x_train_ridge, x_test_ridge, y_train_ridge, y_test_ridge = train_test_split(x, y, test_size = 0.2, random_state = 0)
x_train_ridge.shape, x_test_ridge.shape, y_train_ridge.shape, y_test_ridge.shape

((5148, 3), (1287, 3), (5148, 1), (1287, 1))

In [35]:
# Definir el rango de variación de los parámetros
param_range = {'alpha': np.arange(0.01, 5.1, 0.01),
               'max_iter': range(1,10,1),
               'solver': ['auto', 'svd','cholesky','lsqr','sparce_cg','sag','saga','lbfsg']}

#Veamos los rangos a usar
print(list(param_range['alpha']))
print(list(param_range['max_iter']))
print(list(param_range['solver']))

[0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.06999999999999999, 0.08, 0.09, 0.09999999999999999, 0.11, 0.12, 0.13, 0.14, 0.15000000000000002, 0.16, 0.17, 0.18000000000000002, 0.19, 0.2, 0.21000000000000002, 0.22, 0.23, 0.24000000000000002, 0.25, 0.26, 0.27, 0.28, 0.29000000000000004, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35000000000000003, 0.36000000000000004, 0.37, 0.38, 0.39, 0.4, 0.41000000000000003, 0.42000000000000004, 0.43, 0.44, 0.45, 0.46, 0.47000000000000003, 0.48000000000000004, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.5700000000000001, 0.5800000000000001, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.6900000000000001, 0.7000000000000001, 0.7100000000000001, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.8200000000000001, 0.8300000000000001, 0.8400000000000001, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.9400000000000001, 0.9500000000000001, 0.9600000000000001, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 

In [36]:
# Definir el optimizador
rsearch_ridge = GridSearchCV(Ridge(random_state = 0),
                             param_range,
                             n_jobs = -1 , #Usa todos los nucleos
                             verbose = 4,
                             cv = 4)
#Optimizamos
rsearch_ridge.fit(x_train_ridge, y_train_ridge)

Fitting 4 folds for each of 36648 candidates, totalling 146592 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
36648 fits failed out of a total of 146592.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18324 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 1130, in fit
    return super().fit(X, y, sample_weight=sample_weight)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py", line 889, in fit
    self.coef_, self.n_iter_ = _ridge_r

GridSearchCV(cv=4, estimator=Ridge(random_state=0), n_jobs=-1,
             param_grid={'alpha': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.5...
       4.52, 4.53, 4.54, 4.55, 4.56, 4.57, 4.58, 4.59, 4.6 , 4.61, 4.62,
       4.63, 4.64, 4.65, 4.66, 4.67, 4.68, 4.69, 4.7 , 4.71, 4.72, 4.73,
       4.74, 4.75, 4.76, 4.77, 4.78, 4.79, 4.8 , 4.81, 4.82, 4.83, 4.84,
       4.85, 4.86, 4.87, 4.88, 4.89, 4.9 , 4.91, 4.92, 4.93, 4.94, 4.95,
       4.96, 4.97, 4.98, 4.99, 5.  , 5.01, 5.02, 5.03, 5.04, 5.05, 5.06,
       5.07, 5.08, 5.09]),
                         'max_iter': range(1, 10),
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparce_cg', 'sag', 'saga', 'lbfsg']},
             verbose=4)

In [37]:
print(f"R2 Promedio = {rsearch_ridge.best_score_}")
print(f"Mejor Modelo = {rsearch_ridge.best_estimator_}")
print(f"Mejores Parámetros = {rsearch_ridge.best_params_}")

R2 Promedio = 0.1093204152619458
Mejor Modelo = Ridge(alpha=0.01, max_iter=4, random_state=0, solver='saga')
Mejores Parámetros = {'alpha': 0.01, 'max_iter': 4, 'solver': 'saga'}


In [39]:
# Definimos el modelo
ridge = rsearch_ridge.best_estimator_
#Entrenamos el modelo
ridge.fit(x_train_ridge,y_train_ridge)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Ridge(alpha=0.01, max_iter=4, random_state=0, solver='saga')

In [43]:
# Validación visual
y_pred_test_ridge = ridge.predict(x_test_ridge)
print(y_pred_test_ridge[0:5])
y_test_ridge.head()

[[ 964050.63370797]
 [ 903037.88882178]
 [1176982.16624763]
 [ 851328.70837409]
 [1052173.42010985]]


,Weekly_Sales
3949,1220984.94
4555,1246322.44
2235,505918.21
5204,527572.25
3524,719235.07


__Evaluamos el modelo:__

In [44]:
# Obtieniento el R2 de train
R2_train_ridge = ridge.score(x_train_ridge, y_train_ridge)
print("R2 train de Ridge = {:.2f}".format(R2_train_ridge))

# Obtieniento el R2 de test
R2_test_ridge = ridge.score(x_test_ridge, y_test_ridge)
print("R2 test de Ridge = {:.2f}".format(R2_test_ridge))

# Diferencia
dif_ridge = '{:.4}%'.format(np.abs(R2_train_ridge - R2_test_ridge)*100)
print("Diferencia = ",dif_ridge)

R2 train de Ridge = 0.11
R2 test de Ridge = 0.14
Diferencia =  3.125%


_¡El modelo __no__ tiene overfitting!_

__Precisión del modelo:__

In [45]:
# Obtener el y_pred_train
y_pred_train_ridge = ridge.predict(x_train_ridge)

# Calculamos los errores de train
mape_train_ridge = mape(y_train_ridge, y_pred_train_ridge)*100
mse_train_ridge = mse(y_train_ridge, y_pred_train_ridge)
rmse_train_ridge = np.sqrt(mse_train_ridge)

# 3. Calculamos los errores de test
mape_test_ridge = mape(y_test_ridge, y_pred_test_ridge)*100
mse_test_ridge = mse(y_test_ridge, y_pred_test_ridge)
rmse_test_ridge = np.sqrt(mse_test_ridge)

print("MAPE de train de Ridge = {:.2}%".format(mape_train_ridge))
print("MSE de train de Ridge = {:.2}".format(mse_train_ridge))
print("RMSE de train de Ridge = {:.2}".format(rmse_train_ridge))
print("")
print("MAPE de test de Ridge = {:.2}%".format(mape_test_ridge))
print("MSE de test de Ridge = {:.2}".format(mse_test_ridge))
print("RMSE de test de Ridge = {:.2}".format(rmse_test_ridge))

MAPE de train de Ridge = 6.2e+01%
MSE de train de Ridge = 4.4e+05
RMSE de train de Ridge = 6.6e+02

MAPE de test de Ridge = 6e+01%
MSE de test de Ridge = 4.3e+05
RMSE de test de Ridge = 6.5e+02


---
### Ventajas y desventajas de la Regresión Ridge
_Ventajas:_
1. Puede tratar con la multicolinealidad
2. Reduce el Overfitting
3. Produce resultados con menos varianza

_Desventajas:_
1. Aumenta el sesgo del modelo.
2. La selección del alpha necesita ser metódico y precisa
3. Interpretación del modelo baja

---
### Campo en dónde se utiliza más la Regresión Ridge
---

La regression Ridge es de extrema ayuda cuando se tiene un dataset con amplia cantidad de variables o columnas, ya que su funcionalidad es prevenir o solucionar cualquier problema de multicolinealidad. Gracias a esto, es posible ingresar todas las variables que tengamos ya que de todos modos, el modelo buscará penalizar la multicolinealidad al modificar los coeficientes. Ejemplos de campos de utilización puede ser los mercados en línea que recolectan muchas variables sobre 1 consumidor ya que así se ahorran el tiempo de tener que modificar cualquier selección de variables

# ElasticNet

In [48]:
from sklearn.linear_model import ElasticNet

In [49]:
df_elastic = pd.read_csv("Walmart.csv")
df_elastic['date_num'] = df_ridge['Date'].astype('category').cat.codes

In [51]:
x_elastic = df_elastic.drop(columns=['Weekly_Sales','Date'])
y_elastic = df_ridge['Weekly_Sales']
print(x_elastic.shape)
print(y_elastic.shape)

(6435, 7)
(6435,)


In [53]:
#Definir el rango de variación de los parámetros
param_elastic = {'alpha': [0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,2,3,4,5],
               'l1_ratio': [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1]}

#Veamos los rangos a usar
print(list(param_elastic['alpha']))
print(list(param_elastic['l1_ratio']))

[0.01, 0.03, 0.05, 0.07, 0.1, 0.3, 0.5, 0.7, 1, 2, 3, 4, 5]
[0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]


In [54]:
#Definir el optimizador
rsearch_elastic = GridSearchCV(ElasticNet(random_state=0),
                             param_range,
                             n_jobs=-1 , #Usa todos los nucleos
                             verbose=4,
                             cv=3)
#Optimizamos
rsearch_elastic.fit(x_train_ridge,y_train_ridge)

Fitting 3 folds for each of 36648 candidates, totalling 109944 fits


ValueError: Invalid parameter 'solver' for estimator ElasticNet(alpha=0.01, max_iter=1, random_state=0). Valid parameters are: ['alpha', 'copy_X', 'fit_intercept', 'l1_ratio', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'].

In [ ]:
print(f"R2 Promedio = {rsearch_elastic.best_score_}")
print(f"Mejor Modelo = {rsearch.best_elastic_estimator_}")
print(f"Mejores Parámetros = {rsearch_elastic.best_params_}")

In [ ]:
#Definimos el modelo de RLS
elastic = rsearch_elastic.best_estimator_
#Entrenamos el modelo
elastic.fit(x_train_elastic, y_train_elastic)

In [ ]:
#Validación visual
y_pred_test_elastic = elastic.predict(x_test_elastic)
print(y_pred_test_elastic[0:5])
y_test_elastic.head()

In [ ]:
#Obtener el R2 de train
R2_train_elastic = elastic.score(x_train_elastic, y_train_elastic)
print('R2_train = {:.2f}'.format(R2_train_elastic))

#Obtener el R2 de test
R2_test_elastic = elastic.score(x_test_elastic, y_test_elastic)
print('R2_test = {:.2f}'.format(R2_test_elastic))

dif_elastic = np.abs(R2_train_elastic - R2_test_elastic)*100

#Obtener la diferencia
print('Diferencia = {:.4}%'.format(np.abs(R2_train_elastic - R2_test_elastic)*100))v

In [ ]:
#1. Obtener el dato que nos falta
y_pred_train_elastic = elastic.predict(x_train_ridge)

#2. Calculamos los errores
#Errores de train
mape_train_elastic = mean_absolute_percentage_error(y_train_elastic,y_pred_train_elastic)*100
mse_train_elastic = mean_absolute_error(y_train_elastic, y_pred_train_elastic)
rmse_train_elastic = np.sqrt(mse_train_elastic)

#Errores de test
mape_test_elastic = mean_absolute_percentage_error(y_test_ridge,y_pred_test_elastic)*100
mse_test_elastic = mean_absolute_error(y_test_elastic, y_pred_test_elastic)
rmse_test_elastic = np.sqrt(mse_test_elastic)

print('MAPE_train = {:.4}%'.format(mape_train_elastic))
print('MSE_train = {:.4}'.format(mse_train_elastic))
print('RMSE_train = {:.4}'.format(rmse_train_elastic))
print()
print('MAPE_test = {:.4}%'.format(mape_test_elastic))
print('MSE_test = {:.4}'.format(mse_test_elastic))
print('RMSE_test = {:.4}'.format(rmse_test_elastic))

---
### Ventajas y desventajas de la Regresión ElasticNet
_Ventajas:_
1. Combinación entre modelos Lasso y Ridge

_Desventajas:_
1. Alta necesidad de procesamiento de datos

---
### Campo en dónde se utiliza más la RegresiónElasticNet
---

El modelo de regresión ElasticNet utiliza una combinación de los modelos Lasso y Ridge, por lo que es util para realizar subgrupos y penalizar multicolinealidad. Sin embargo, este modelo necesita ser utilizado en grandes proyectos que tengan la inversión suficiente para poder soportar este nivel de procesamiento

Tabla de comparaciones
=

In [ ]:
comparisson = {'MLR': [R2_train, R2_test, dif, mape_train, mse_train, rmse_train, mape_test, mape_test, rmse_test], 
               'Lasso': [R2_train_lasso, R2_test_lasso, dif_lasso, mape_train_lasso, mse_train_lasso, rmse_train_lasso, mape_test_lasso, mape_test_lasso, rmse_test_lasso],
               'Ridge': [R2_train_ridge, R2_test_ridge, dif_ridge, mape_train_ridge, mse_train_ridge, rmse_train_ridge, mape_test_ridge, mape_test_ridge, rmse_test_ridge],
               'Elastic Net': [R2_train_Elastic, R2_test_Elastic, dif_elastic, mape_train_elastic, mse_train_elastic, rmse_train_elastic, mape_test_elastic, mape_test_elastic, rmse_test_elastic]}
pd.DataFrame(data = comparisson, index = ['R2 de train', 'R2 de test', 'Diferencia R2',
                                          'MAPE de train', 'MSE de train', 'RMSE de train',
                                          'MAPE de test', 'MSE de test', 'RMSE de test'])

## Referencias:
Eculedian (2019). _Pros and cons of common Machine Learning algorithms_ medium.com link=https://medium.com/@eculidean/pros-and-cons-of-common-machine-learning-algorithms-45e05423264f
Ohri, A. (2021). _LASSO Regression: A Complete Understanding (2021)_. Unext. Recuperado de: https://www.jigsawacademy.com/blogs/ai-ml/lasso-regression#:~:text=Lasso%20regression%20is%20also%20called,helps%20to%20increase%20model%20interpretation.
Joaquin R. (2020). _Regularización Ridge, Lasso y Elastic Net con Python._ cienciadedatos.net Recuperado de: https://www.cienciadedatos.net/documentos/py14-ridge-lasso-elastic-net-python.html